In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
compteur = 0
for dirname, _, filenames in os.walk('/kaggle/input/siim-covid19-detection/train'):
    for filename in filenames:
      compteur += 1
compteur

In [ ]:
liste = []
for dirname, _, filenames in os.walk('/kaggle/input/siim-covid19-detection/test'):
    for filename in filenames:
        liste.append(os.path.join(dirname, filename))
len(liste)

In [ ]:
liste[:10]

In [ ]:
compteur = 0
for dirname in os.walk('/kaggle/input/siim-covid19-detection/test'):
    compteur += 1
compteur

In [ ]:
df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
df_box = pd.read_csv('/kaggle/input/siim-covid19-detection/train_image_level.csv')
df_classif = pd.read_csv('/kaggle/input/siim-covid19-detection/train_study_level.csv')
df_classif['id'] = df_classif['id'].apply(lambda x: f'{x[:12]}')
df_classif.rename(columns = {'id':'StudyInstanceUID'}, inplace = True)
data = pd.merge(df_box, df_classif, on='StudyInstanceUID')
data['id'] = data['id'].apply(lambda x: x.split('_')[0])

data[ data['Negative for Pneumonia'] == 1 ] # pas de bbox dans ces cas là

In [ ]:
len(df) # TOUT LE DOSSIER TEST: 1263 études (1263 dossiers dans le dossier test) + toutes les images qu'ils contiennent = 2475

In [ ]:
len(data)

In [ ]:
import glob
# Trouver tous les chemins vers les fichiers qui finissent par .jpg
liste = glob.glob('/kaggle/input/siim-covid19-detection/train/*/*/*.dcm')
liste[:10]

df2 = pd.DataFrame(liste, columns = ['URL'])
df2['id'] = df2['URL'].apply(lambda x : os.path.split(x)[-1])
df2['id'] = df2['id'].apply(lambda x : x.split('.')[0])
data['id'] = data['id'].apply(lambda x : x.split('_')[0])
FinalDf = pd.merge(data, df2, on='id')
FinalDf.head()

In [ ]:
import json
def read_box(box):
    try:
        return json.loads(box.replace('\'', '\"'))
    except AttributeError:
        return []
    
FinalDf['bbox_nbr'] = FinalDf['boxes'].apply(lambda x : len(read_box(x)) ).astype(int)
FinalDf.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(6, 3))
sns.barplot(x = FinalDf['bbox_nbr'].value_counts().index, y = FinalDf['bbox_nbr'].value_counts().values, palette="husl")
plt.ylabel('Image number')
plt.xlabel('Bbox number')

Etant donné que les radios avec 3 ou 4 bbox sont largement minoritaire nous allons considérer uniquement celles avec 0, 1 ou 2 bbox dans un premier temps

In [ ]:
FinalDf_0bbox = FinalDf[ FinalDf['bbox_nbr'] == 0 ]
FinalDf_0bbox = FinalDf_0bbox.reset_index()
FinalDf_0bbox = FinalDf_0bbox.drop(['index'], axis=1)

FinalDf_1bbox = FinalDf[ FinalDf['bbox_nbr'] == 1 ]
FinalDf_1bbox = FinalDf_1bbox.reset_index()
FinalDf_1bbox = FinalDf_1bbox.drop(['index'], axis=1)

FinalDf_2bbox = FinalDf[ FinalDf['bbox_nbr'] == 2 ]
FinalDf_2bbox = FinalDf_2bbox.reset_index()
FinalDf_2bbox = FinalDf_2bbox.drop(['index'], axis=1)

# FinalDf_3bbox = FinalDf[ FinalDf['bbox_nbr'] == 3] on va pas considérer ceux-là dans un 1er temps
FinalDf_2bbox.head()

In [ ]:
# coordonnées bounding box 1
FinalDf_1bbox['x1 bb1'] = FinalDf_1bbox['label'].apply(lambda x : x.split(' ')[2]).astype(float)
FinalDf_1bbox['y1 bb1'] = FinalDf_1bbox['label'].apply(lambda x : x.split(' ')[3]).astype(float)
FinalDf_1bbox['x2 bb1'] = FinalDf_1bbox['label'].apply(lambda x : x.split(' ')[4]).astype(float)
FinalDf_1bbox['y2 bb1'] = FinalDf_1bbox['label'].apply(lambda x : x.split(' ')[5]).astype(float)

FinalDf_1bbox['xmoy bb1'] = (FinalDf_1bbox['x1 bb1'] + FinalDf_1bbox['x2 bb1']) /2
FinalDf_1bbox['ymoy bb1'] = (FinalDf_1bbox['y1 bb1'] + FinalDf_1bbox['y2 bb1']) /2
FinalDf_1bbox['w bb1'] = FinalDf_1bbox['x2 bb1'] - FinalDf_1bbox['x1 bb1']
FinalDf_1bbox['h bb1'] = FinalDf_1bbox['y2 bb1'] - FinalDf_1bbox['y1 bb1']

FinalDf_1bbox.head(1)

In [ ]:
# coordonnées bounding box 1
FinalDf_2bbox['x1 bb1'] = FinalDf_2bbox['label'].apply(lambda x : x.split(' ')[2]).astype(float)
FinalDf_2bbox['y1 bb1'] = FinalDf_2bbox['label'].apply(lambda x : x.split(' ')[3]).astype(float)
FinalDf_2bbox['x2 bb1'] = FinalDf_2bbox['label'].apply(lambda x : x.split(' ')[4]).astype(float)
FinalDf_2bbox['y2 bb1'] = FinalDf_2bbox['label'].apply(lambda x : x.split(' ')[5]).astype(float)

FinalDf_2bbox['xmoy bb1'] = (FinalDf_2bbox['x1 bb1'] + FinalDf_2bbox['x2 bb1']) /2
FinalDf_2bbox['ymoy bb1'] = (FinalDf_2bbox['y1 bb1'] + FinalDf_2bbox['y2 bb1']) /2
FinalDf_2bbox['w bb1'] = FinalDf_2bbox['x2 bb1'] - FinalDf_2bbox['x1 bb1']
FinalDf_2bbox['h bb1'] = FinalDf_2bbox['y2 bb1'] - FinalDf_2bbox['y1 bb1']

# coordonnées bounding box 2
FinalDf_2bbox['x1 bb2'] = FinalDf_2bbox['label'].apply(lambda x : x.split(' ')[8]).astype(float)
FinalDf_2bbox['y1 bb2'] = FinalDf_2bbox['label'].apply(lambda x : x.split(' ')[9]).astype(float)
FinalDf_2bbox['x2 bb2'] = FinalDf_2bbox['label'].apply(lambda x : x.split(' ')[10]).astype(float)
FinalDf_2bbox['y2 bb2'] = FinalDf_2bbox['label'].apply(lambda x : x.split(' ')[11]).astype(float)

FinalDf_2bbox['xmoy bb2'] = (FinalDf_2bbox['x1 bb2'] + FinalDf_2bbox['x2 bb2']) /2
FinalDf_2bbox['ymoy bb2'] = (FinalDf_2bbox['y1 bb2'] + FinalDf_2bbox['y2 bb2']) /2
FinalDf_2bbox['w bb2'] = FinalDf_2bbox['x2 bb2'] - FinalDf_2bbox['x1 bb2']
FinalDf_2bbox['h bb2'] = FinalDf_2bbox['y2 bb2'] - FinalDf_2bbox['y1 bb2']

FinalDf_2bbox.head(1)

In [ ]:
df_intermediaire = pd.concat([FinalDf_0bbox, FinalDf_1bbox], axis=0)
data = pd.concat([df_intermediaire, FinalDf_2bbox], axis=0)

dico = {'Negative for Pneumonia': 'Negative',
        'Typical Appearance': 'Typical',
        'Indeterminate Appearance': 'Ind',
        'Atypical Appearance': 'Atypical'}
data = data.rename(dico, axis = 1) 
data.tail()

In [ ]:
FinalDf = data.fillna(0)
FinalDf = FinalDf .reset_index()
cols_to_drop = ['boxes', 'label', 'StudyInstanceUID']
FinalDf = FinalDf.drop(cols_to_drop, axis=1)
FinalDf.info()

In [ ]:
len(FinalDf) # 6334 images moins celles qui ont 3 ou 4 bbox

In [ ]:
FinalDf.head()

In [ ]:
!pip install tensorflow-io

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_io as tfio

def load_image(pathname):
    image_bytes = tf.io.read_file(pathname)
    image = tfio.image.decode_dicom_image(image_bytes, dtype=tf.uint16)
    return image

im = load_image(FinalDf['URL'][0]) 

plt.figure(figsize=(5, 5))
plt.imshow(np.squeeze(im.numpy()), cmap='gray');

In [ ]:
print("Tensorflow version " + tf.__version__)

In [ ]:
def show_two_bounding_box(im, bbox1, bbox2, color='r'):
    # Signification de bbox
    x11, y11, x21, y21 = bbox1
    x12, y12, x22, y22 = bbox2

    # Afficher l'image
    plt.figure(figsize=(5, 5))
    plt.imshow(np.squeeze(im.numpy()), cmap='gray')
    
    # Afficher les bounding box
    plt.plot([x11,x21,x21,x11,x11],[y11,y11,y21,y21,y11],"r")
    plt.plot([x12,x22,x22,x12,x12],[y12,y12,y22,y22,y12],"r")

idx = 3 # 
# Array de l'image
im = load_image(FinalDf_2bbox.URL[idx]) 
# Coordonnées de la bounding box
bbox1 = FinalDf_2bbox[['x1 bb1', 'y1 bb1', 'x2 bb1', 'y2 bb1']].values[idx]
bbox2 = FinalDf_2bbox[['x1 bb2', 'y1 bb2', 'x2 bb2', 'y2 bb2']].values[idx]
# Afficher l'image ainsi que la bounding box
show_two_bounding_box(im, bbox1, bbox2)

In [ ]:
idx = 2045 # len(FinalDf_0bbox) = 2040 donc pour cet indice on est dans une entrée qui a une seule bbox
# Array de l'image
im = load_image(FinalDf.URL[idx]) 
# Coordonnées de la bounding box
bbox1 = FinalDf[['x1 bb1', 'y1 bb1', 'x2 bb1', 'y2 bb1']].values[idx]
bbox2 = FinalDf[['x1 bb2', 'y1 bb2', 'x2 bb2', 'y2 bb2']].values[idx]
# Afficher l'image ainsi que la bounding box
show_two_bounding_box(im, bbox1, bbox2)

In [ ]:
idx = 7 # 0 bbox pour les 1eres entrées
# Array de l'image
im = load_image(FinalDf.URL[idx]) 
# Coordonnées de la bounding box
bbox1 = FinalDf[['x1 bb1', 'y1 bb1', 'x2 bb1', 'y2 bb1']].values[idx]
bbox2 = FinalDf[['x1 bb2', 'y1 bb2', 'x2 bb2', 'y2 bb2']].values[idx]
# Afficher l'image ainsi que la bounding box
show_two_bounding_box(im, bbox1, bbox2)

In [ ]:
def load_resize_image(pathname, resize=(256,256)):
    image_bytes = tf.io.read_file(pathname)
    image = tfio.image.decode_dicom_image(image_bytes, dtype=tf.uint16)
    image = tf.image.resize(im, resize)
    return image.numpy() # enlever le .numpy() si c'est mieux de garder un tensor

im = load_resize_image(FinalDf['URL'][0]) 
im = im.reshape(im.shape[1], im.shape[2]) # shape = 256x256
# For grayscale images, the pixel value is typically an 8-bit data value (with a range of 0 to 255) 
# or a 16-bit data value (with a range of 0 to 65535)
np.max(im) # -> max pixel value  (si jamais on veut normaliser) 

In [ ]:
from sklearn.model_selection import train_test_split
X_train_path, X_test_path, y_train, y_test = train_test_split(FinalDf.URL.values, 
                                             FinalDf[['x1 bb1', 'y1 bb1', 'x2 bb1', 'y2 bb1', 
                                                      'Negative', 'Typical', 'Ind', 'Atypical',
                                                      'x1 bb2', 'y1 bb2', 'x2 bb2', 'y2 bb2']].values, 
                                                       train_size=0.9, random_state=1234)

In [ ]:
y_train[10] # 2 bbox 

In [ ]:
y_train[10][8:12]

In [ ]:
filepath = X_train_path[10]
im = load_image(filepath) 
bbox1 = y_train[10][0:4]
bbox2 = y_train[10][8:12]
show_two_bounding_box(im, bbox1, bbox2)

In [ ]:
def load_image_convert_target(pathname, pouet):

    image_bytes = tf.io.read_file(pathname)
    image = tfio.image.decode_dicom_image(image_bytes, dtype=tf.uint16)
    img_shape = image.numpy().shape 
    resize = (224,224)
    image = tf.image.resize(image, resize)
    image = tf.image.grayscale_to_rgb(image)

    # on initialise une matrice de zero de dimension 8x8 x5
    output_shape = (7, 7)
    y_target = np.zeros([output_shape[0], output_shape[1], 9])
    
    lx = 1/output_shape[1] # lx = 1/8eme
    ly = 1/output_shape[0] # ly = 1/8eme
    
    # on modifie dans la matrice les 5 valeurs del'array correspondant à la case contenant le centre de la bbox
    for x1, y1, x2, y2, a, b, c, d in [pouet[:8]]:

        # normalisation entre 0 et 1:
        x1 = x1/img_shape[2]
        y1 = y1/img_shape[1]
        x2 = x2/img_shape[2]
        y2 = y2/img_shape[1]
        
        x = (x1+x2)/2 # x milieu bbox
        y = (y1+y2)/2 # y milieu bbox
        w = x2 - x1
        h = y2 - y1
        
        idx_x = int(x//lx) # numéro case abcisse
        idx_y = int(y//ly) # numéro case ordonnée

        y_target[idx_y, idx_x, 0] = 1                      # p true object case
        y_target[idx_y, idx_x, 1] = 2*(x/lx - (idx_x+0.5)) # x
        y_target[idx_y, idx_x, 2] = 2*(y/ly - (idx_y+0.5)) # y
        y_target[idx_y, idx_x, 3] = w                      # w
        y_target[idx_y, idx_x, 4] = h                      # h
        y_target[idx_y, idx_x, 5] = a                      # Neg for Pneumo (0ou1)
        y_target[idx_y, idx_x, 6] = b                      # Typical (0ou1)
        y_target[idx_y, idx_x, 7] = c                      # Ind (0ou1)
        y_target[idx_y, idx_x, 8] = d                      # Atypical (0ou1)
        
    for a, b, c, d, x1, y1, x2, y2 in [pouet[4:]]:

        x1 = x1/img_shape[2]
        y1 = y1/img_shape[1]
        x2 = x2/img_shape[2]
        y2 = y2/img_shape[1]
        
        x = (x1+x2)/2
        y = (y1+y2)/2
        w = x2 - x1
        h = y2 - y1
        
        idx_x = int(x//lx) # numéro case abcisse
        idx_y = int(y//ly) # numéro case ordonnée
        
        y_target[idx_y, idx_x, 0] = 1                      # p true object case
        y_target[idx_y, idx_x, 1] = 2*(x/lx - (idx_x+0.5)) # x    
        y_target[idx_y, idx_x, 2] = 2*(y/ly - (idx_y+0.5)) # y
        y_target[idx_y, idx_x, 3] = w                      # w
        y_target[idx_y, idx_x, 4] = h                      # h
        y_target[idx_y, idx_x, 5] = a                      # Neg for Pneumo (0ou1)
        y_target[idx_y, idx_x, 6] = b                      # Typical (0ou1)
        y_target[idx_y, idx_x, 7] = c                      # Ind (0ou1)
        y_target[idx_y, idx_x, 8] = d                      # Atypical (0ou1)
        
    return image.numpy().reshape([224, 224, 3])/np.max(image.numpy().reshape([224, 224, 3]))*255, y_target.reshape([-1, 9]) # tuple dont la 1ere valeur est l'array de 
                                                                           # l'image et la 2nde est une grille qui aura les 9 valeurs
                                                                           # target de bbox dans les 2 cases concernées


load_image_convert_target(X_train_path[10], y_train[10])[1]
# en partant du principe que la classe est la même pour les 2 bbox

In [ ]:
print("max pixel value image10: ", np.max(load_image_convert_target(X_train_path[10], y_train[10])[0]))
load_image_convert_target(X_train_path[10], y_train[10])[0].shape

In [ ]:
print(load_image_convert_target(X_train_path[10], y_train[10])[1][:,0].shape)
load_image_convert_target(X_train_path[10], y_train[10])[1][:,0] # grille de 64 cases avec 1 quand le centre de la bbox s'y trouve

In [ ]:
def show_two_bounding_box(pathname, cible, color='r'):

    im = load_image(pathname)
    img_shape = np.squeeze(im.numpy()).shape
    resize = (224,224)
    im = tf.image.resize(im, resize)

    x11, y11, x21, y21 = cible[:4][0], cible[:4][1], cible[:4][2], cible[:4][3]
    x12, y12, x22, y22 = cible[8:][0], cible[8:][1], cible[8:][2], cible[8:][3]
    
    x11 = x11/img_shape[1] * 224
    y11 = y11/img_shape[0] * 224
    x21 = x21/img_shape[1] * 224
    y21 = y21/img_shape[0] * 224
    
    x12 = x12/img_shape[1] * 224
    y12 = y12/img_shape[0] * 224
    x22 = x22/img_shape[1] * 224
    y22 = y22/img_shape[0] * 224

    plt.imshow(np.squeeze(im.numpy()))
    plt.plot([x11,x21,x21,x11,x11],[y11,y11,y21,y21,y11],"r")
    plt.plot([x12,x22,x22,x12,x12],[y12,y12,y22,y22,y12],"r")

output_shape = (7,7)
plt.figure(figsize=(12,5))
plt.subplot(121)
show_two_bounding_box(X_train_path[10], y_train[10])
plt.subplot(122)
plt.imshow(load_image_convert_target(X_train_path[10], y_train[10])[1][:,0].reshape(output_shape))

In [ ]:
# !pip uninstall tensorflow -y
# !pip install tensorflow-gpu
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# from tensorflow.python.client import device_lib 
# print(device_lib.list_local_devices())
# !nvidia-smi

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, LeakyReLU, Dropout, Reshape, BatchNormalization
from tensorflow.keras import Sequential

nb_class = 4

# Backbone
efficientNet = EfficientNetB0(include_top=False, input_shape=(224,224,3)) 

# Freeze the blackbone
for layer in efficientNet.layers:
    layer.trainable = False

# Définition de la partie encoder
model = Sequential()
# Feature extration part
model.add(efficientNet) 
model.add(Reshape([-1, 1280]))
# Regression Part
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5 + nb_class))
model.summary()


In [ ]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train_path, y_train))

In [ ]:
dataset_train = dataset_train.shuffle(100000).map(lambda x, y : tf.py_function(load_image_convert_target, [x, y], [tf.float32, tf.float32]), num_parallel_calls=-1).batch(32)

In [ ]:
X_t, y_t = next(iter(dataset_train))
X_t[0].numpy().shape

In [ ]:
X_t.numpy().shape

In [ ]:
y_pred_raw = model.predict(X_t)
y_pred_raw.shape

In [ ]:
def transform_netout(y_pred_raw):
    
    y_pred_conf = tf.sigmoid(y_pred_raw[...,0:1]) # outputs sigmoïde [0,1] et notre p_true doit valoir 0 ou 1 
    y_pred_xy = (tf.nn.tanh(y_pred_raw[...,1:3])) # pour avoir des valeurs comprises entre -1 et 1 avec tanh --> rappel: chaque case de la grille devient un nouveau repère dont l'origine est centrée: le max des abcisses et des ordonnées est 1 et le min -1
    y_pred_wh = (tf.sigmoid(y_pred_raw[...,3:5])) # parce que la largeur et la hauteur de la bbox peuvent valoir 0 au min et 1 au max
    y_pred_class = tf.nn.softmax(y_pred_raw[..., 5:])  # pour avoir des probas comprises entre 0 et 1 pour chacune des 4 classes VERIFIER!!

    return tf.concat([y_pred_conf, y_pred_xy, y_pred_wh, y_pred_class], -1)

y_pred = transform_netout(y_pred_raw)
y_pred

In [ ]:
for (X_b, y_b) in dataset_train.take(count=2):
    print('Shape of the batch X :', X_b.numpy().shape)
    print('Target elements of the batch :', y_b.numpy().shape, '\n')

In [ ]:
def class_loss(y_true, y_pred):
    y_true_conf = y_true[:,:,0]
    y_true_class = y_true[:,:,5:] # on est déjà en one hot 
    y_pred_class = y_pred[:,:,5:]
    # Calcul de la fonction de perte
    class_loss = tf.reduce_sum(y_true_conf*tf.reduce_sum(tf.square(y_true_class - y_pred_class), axis=-1), axis=-1)
    # mettre une softmax plutôt?
    return class_loss # vaudra zéro pour toutes les cases sans bbox. zéro s'il yen a une et que la pred est correcte et 1 sinon (-1 au carré)

class_loss(y_t, y_pred)

In [ ]:
def coord_loss(y_true, y_pred):
    y_true_conf = y_true[...,0]
    y_true = y_true[...,1:5]
    y_pred= y_pred[...,1:5]
    return tf.reduce_sum(tf.reduce_sum(tf.square(y_true- y_pred),axis=-1)*y_true_conf, axis=-1)

coord_loss(y_t, y_pred)

In [ ]:
lambda_noobj = 0.5
def object_loss(y_true, y_pred):
    # x and y loss for real object
    y_true_p = y_true[...,0]
    y_pred_p = y_pred[...,0]
    return tf.reduce_sum((lambda_noobj + (1-lambda_noobj)*y_true_p)*tf.square(y_true_p - y_pred_p), axis=-1)

# Object loss for the prediction X_t
object_loss(y_t, y_pred)

In [ ]:
lambda_coord = 5
lambda_object = 1
# lamda_class = ?

def global_loss(y_true, y_pred):
    # Convert input
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_pred = transform_netout(y_pred)
    loss_class = class_loss(y_true, y_pred)
    loss_coordinate = coord_loss(y_true, y_pred)
    loss_object = object_loss(y_true, y_pred)
    return lambda_object*loss_object + lambda_coord*loss_coordinate + loss_class #ajouter lambda class

# Global loss for the prediction X_t
global_loss(y_t, y_pred)

In [ ]:
#model_save_path = '/kaggle/input/modeltest1/model_test1_12epochs.joblib'
#weights_path = '/kaggle/input/modeltest1/Auto_save_model_correctedNormalize_14_epochs.h5'

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import callbacks

autosave_path = "Auto_save_model.h5"

checkpoint = callbacks.ModelCheckpoint(autosave_path, 
                                       monitor = 'loss',
                                       save_best_only = True,
                                       save_weights_only = True,
                                       mode = 'min',
                                       save_freq = 'epoch')


# Callback to reduce automatically the learning rate.
lr_plateau = callbacks.ReduceLROnPlateau(monitor = 'loss',
                                         patience=5,
                                         factor=0.1,
                                         verbose=2,
                                         mode='min')


model.compile(optimizer=Adam(1e-3), loss = global_loss)
#model.load_weights(weights_path)
epochs = 10
# history = model.fit(dataset_train, epochs = epochs, callbacks = [lr_plateau, checkpoint])

In [ ]:
#plt.figure(figsize=(12,10))
#plt.plot(history.history['loss'])
#plt.title("Loss evolution test model 1")
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.savefig("loss_evol_8ep.png");

In [ ]:
# model_save_path = "model" + str(epochs) + ".joblib"
# model.save( model_save_path)

In [ ]:
model_10ep_path = "../input/10epochstrain/model10.joblib"
model_trained_10 = tf.keras.models.load_model(model_10ep_path, custom_objects = {"global_loss":global_loss})
weights_path = "../input/10epochstrain/Auto_save_model.h5"
model_trained_10.load_weights(weights_path)

In [ ]:
def process_prediction(y_pred, threshold=0.8, grid=(7, 7)):
    ### Return the coordinates of the x, y, w and h in out_image unity
    if type(y_pred) == np.ndarray:
        pred = y_pred.reshape((grid[0], grid[1], 9))
    else:
        pred = y_pred.numpy().reshape((grid[0], grid[1], 9))
    idx_y, idx_x = np.where(pred[..., 0] > threshold)
    lx = 1 / grid[0]
    ly = 1 / grid[1]
    
    x = (pred[idx_y, idx_x, 1] /2 + (idx_x + 0.5) )* lx # 2*(x/lx - (idx_x+0.5))à l'envers
    y = (pred[idx_y, idx_x, 2] /2 + (idx_y + 0.5) )* ly
    w = pred[idx_y, idx_x, 3]
    h = pred[idx_y, idx_x, 4]
    x,y,w,h = x*224,y*224,w*224,h*224

    c = np.argmax(pred[idx_x, idx_y, 5:], axis=1)
    return x - (w / 2), y - (h / 2), x + (w / 2),  y + (h / 2), c

In [ ]:
def show_two_bboxFROMpred(im, coord_pred, color='r'):
    x1, y1, x2, y2 = coord_pred[:4]
    plt.imshow(im/255)
    plt.plot([x1,x2,x2,x1,x1],[y1,y1,y2,y2,y1],color)

AFFICHAGE DES PREDICTIONS FAITES AVANT L'ENTRAINEMENT DU MODELE

In [ ]:
plt.figure(figsize=(10, 8))

plt.subplot(1, 3, 1)
plt.title('All TRUE bboxes for the batch of 32 img ')
for i in range(32):
    show_two_bboxFROMpred(X_t[i], process_prediction(y_t[i]))
 
plt.subplot(1, 3, 2)
plt.title('All pred - seuil 0.8 ')
for i in range(32):
    show_two_bboxFROMpred(X_t[i], process_prediction(y_pred[i]))
    
plt.subplot(1, 3, 3)
plt.title('All pred - seuil 0.5 ')
for i in range(32):
    show_two_bboxFROMpred(X_t[i], process_prediction(y_pred[i], threshold=0.5))

APRES 10 EPOCHS

In [ ]:
y_t_pred_raw_10ep = model_trained_10.predict(X_t)
y_t_pred_10ep = transform_netout(y_t_pred_raw_10ep)

plt.figure(figsize=(10, 8))

plt.subplot(1, 3, 1)
plt.title('All TRUE bboxes for the batch of 32 img ')
for i in range(32):
    show_two_bboxFROMpred(X_t[i], process_prediction(y_t[i]))
 
plt.subplot(1, 3, 2)
plt.title('All pred - seuil 0.8 ')
for i in range(32):
    show_two_bboxFROMpred(X_t[i], process_prediction(y_t_pred_10ep[i]))
    
plt.subplot(1, 3, 3)
plt.title('All pred - seuil 0.5 ')
for i in range(32):
    show_two_bboxFROMpred(X_t[i], process_prediction(y_t_pred_10ep[i], threshold=0.5))

In [ ]:
for i in range(12):
 plt.figure(figsize=(10, 8))

 plt.subplot(1, 3, 1)
 plt.title('img '+ str(i) +' TRUE')
 show_two_bboxFROMpred(X_t[i], process_prediction(y_t[i]), color='blue') 

 plt.subplot(1, 3, 2)
 plt.title('img '+ str(i) + ' PRED - seuil 0.8 ')
 show_two_bboxFROMpred(X_t[i], process_prediction(y_t_pred_10ep[i]))

 plt.subplot(1, 3, 3)
 plt.title('img '+ str(i) + ' PRED - seuil 0.5 ')
 show_two_bboxFROMpred(X_t[i], process_prediction(y_t_pred_10ep[i], threshold=0.5))

In [ ]:
from tqdm import tqdm

def load_Data( data, targets ):
    img_list=[]
    label_list=[]
    for i in tqdm(range(len(data))):
        img_list.append( load_image_convert_target(data[i], targets[i])[0] )
        label_list.append( load_image_convert_target(data[i], targets[i])[1] )
        if i%500==0:
            print(i)
    return np.array(img_list), np.array(label_list)
 

dataset_test = load_Data( X_train_path[:100], y_train[:100] ) 

X_train = dataset_test[0]
y_t_train = dataset_test[1]
# y_pred_train = model.predict(X_train)
y_pred_10ep = model_trained_10.predict(X_train)
y_train_pred_10ep = transform_netout(y_pred_10ep)

In [ ]:
def show_two_bounding_boxV1(pathname, target, color='r'):
    
    image_bytes = tf.io.read_file(pathname)
    image = tfio.image.decode_dicom_image(image_bytes, dtype=tf.uint16)
    # Signification de bbox
    x11, y11, x21, y21 = target[:4]
    x12, y12, x22, y22 = target[8:]

    # Afficher l'image
    #plt.figure(figsize=(5, 5))
    plt.imshow(np.squeeze(image.numpy()), cmap='gray')
    
    # Afficher les bounding box
    plt.plot([x11,x21,x21,x11,x11],[y11,y11,y21,y21,y11], color)
    plt.plot([x12,x22,x22,x12,x12],[y12,y12,y22,y22,y12], color)

VERIFICATION QU'APRES LE REDIMENSIONNEMENT ET LA CONVERSION/DECONVERSION DES COORDONNEES EN GRILLES LES BBOX SONT BIEN AU MEME ENDROIT SUR L'IMAGE

In [ ]:
plt.figure(figsize=(15, 6))
for i in range(5):

  plt.subplot(2, 5, i+1)
  plt.title('img '+ str(i+3) +' initial DIM')
  show_two_bounding_boxV1(X_train_path[i+3], y_train[i+3], color='b')

  plt.subplot(2, 5, 6+i)
  plt.title('img '+ str(i+3) +' REDIM')
  show_two_bboxFROMpred(X_train[i+3], process_prediction(y_t_train[i+3]))

In [ ]:
def show_trueANDpred(im, cible, true_cible):
    plt.imshow(im/255)
    
    x1, y1, x2, y2 = cible[:4]
    plt.plot([x1,x2,x2,x1,x1],[y1,y1,y2,y2,y1],"r")
    
    x1, y1, x2, y2 = true_cible[:4]
    plt.plot([x1,x2,x2,x1,x1],[y1,y1,y2,y2,y1],"b")

AFFICHAGE DES PREDICITONS APRES 15 EPOCHS D'ENTRAINEMENT

In [ ]:
plt.figure(figsize=(15, 50))
for i in range(100):
  plt.subplot(20, 5, i+1)
  show_trueANDpred(X_train[i], process_prediction(y_train_pred_10ep[i], threshold=0.5), process_prediction(y_t_train[i]))

In [ ]:
plt.figure(figsize=(15, 50))
for i in range(100):
  plt.subplot(20, 5, i+1)
  show_trueANDpred(X_train[i], process_prediction(y_train_pred_10ep[i], threshold=0.75), process_prediction(y_t_train[i]))

Le modèle n'est pas encore assez entraîné pour prédire des bbox avec une probabilité robuste il a "vu" une grande majorité de cases vides donc il a tendance à prédire qu'il n'y a pas d'objet.

In [ ]:
epochs = 5
history = model_trained_10.fit(dataset_train, epochs = epochs, callbacks = [lr_plateau, checkpoint])

In [ ]:
model_save_path = "model_15ep.joblib"
model_trained_10.save( model_save_path)

In [ ]:
print(VA ARRETER LE SCRIPT)

In [ ]:
def generate_yolo_grid(g):
    c_x = tf.cast(tf.reshape(tf.tile(tf.range(g), [g]), (1, g, g)), 'float32')
    c_y = tf.transpose(c_x, (0,2,1))
    return tf.stack([tf.reshape(c_x, (-1, g*g)), tf.reshape(c_y, (-1, g*g))] , -1)

c_grid = generate_yolo_grid(output_shape[0])


In [ ]:
def proccess_xy(y_true_raw): # y_true shape 64 par 9
    y_true_xy = ((y_true_raw[..., 1:3]+1)/2 + c_grid)/output_shape[0]
    y_true_wh = y_true_raw[..., 3:5]
    y_true_conf = y_true_raw[..., :1]
    y_true_class = y_true_raw[..., 5:]
    return tf.concat([y_true_conf, y_true_xy, y_true_wh, y_true_class], -1) 
# 64 par 9 on a juste changé la 2e et la 3e valeur

In [ ]:
def pred_bboxes(y, threshold=0.8):
    y_xy = tf.cast(y, tf.float32)
    y_xy = tf.expand_dims(y_xy, axis=0) # 1 x 64 x 9
    y_xy = proccess_xy(y_xy)[0] # = proccess_xy(y_xy) mais comme on a rajoute une dimension c'est pour que ce soit bien 64x9
    #return y_xy
    bboxes =  sorted(y_xy.numpy(), key=lambda x: x[0], reverse=True)
    bboxes = np.array(bboxes)
    result = bboxes[bboxes[...,0]>threshold]
    if len(result)== 0:
        return np.zeros((1, 9))
        # return bboxes[[0]]
    return result    

In [ ]:
model_save_path = output_path + "model_test1_10epochs" + ".joblib"
model.save( model_save_path)

In [ ]:
! ls '/kaggle/input/notebookfc4bf591d7/model_test1_10epochs.joblib'

In [ ]:
model2 = tf.keras.models.load_model(model_save_path, custom_objects = {"global_loss":global_loss})

In [ ]:
model2.load_weights(autosave_path)

In [ ]:
history2 = model.fit(dataset_train, epochs = 3, workers=-1,  callbacks = [lr_plateau, checkpoint])

In [ ]:
def load_image_augmentation_convert_target(pathname, cible): 
    
    image_bytes = tf.io.read_file(pathname)
    image = tfio.image.decode_dicom_image(image_bytes, dtype=tf.uint16)
    img_shape = image.numpy().shape 
    resize = (256,256)
    image = tf.image.resize(image, resize)
        
    ############################### bbox1 
    x1 = cible[0]/img_shape[2]
    y1 = cible[1]/img_shape[1]
    x2 = cible[2]/img_shape[2]
    y2 = cible[3]/img_shape[1]
    
    # Find the max translation to keep the object in the image
    tx_max_bbox1 = 256 * (1 - x1)           # abscisse max de l'image shiftee
    tx_min_bbox1 = -256 * (1 - x2)          # abscisse min du shift de l'image shiftee
    ty_max_bbox1 = 256*tf.nn.relu( 1 - y1 ) # ordonne max de l'image shiftee
    ty_min_bbox1 = -256*tf.nn.relu(1 - y2)  # ordonne min de l'image
    
    ############################### bbox2
    x1 = cible[8]/img_shape[2]
    y1 = cible[9]/img_shape[1]
    x2 = cible[10]/img_shape[2]
    y2 = cible[11]/img_shape[1]
    
    # Find the max translation to keep the object in the image
    tx_max_bbox2 = 256 * (1 - x1)           # abscisse max de l'image shiftee
    tx_min_bbox2 = -256 * (1 - x2)          # abscisse min du shift de l'image shiftee
    ty_max_bbox2 = 256*tf.nn.relu( 1 - y1 ) # ordonne max de l'image shiftee
    ty_min_bbox2 = -256*tf.nn.relu(1 - y2)  # ordonne min de l'image
      
    ################### SHIFT FINAL image
    tx = np.random( (np.min(tx_min_bbox1, tx_min_bbox2)), (np.min(tx_max_bbox1, tx_max_bbox2)) ) # horizontal translation
    ty = np.random( (np.min(ty_min_bbox1, ty_min_bbox2)), (np.min(ty_max_bbox1, ty_max_bbox2)) ) # vertical translation
    
    # New coord calcul
    cible[0] += tx/256
    cible[1] += ty/256
    cible[2] += tx/256
    cible[3] += ty/256
    cible[8] += tx/256
    cible[9] += ty/256
    cible[10] += tx/256
    cible[11] += ty/256
    
    im = tf.keras.preprocessing.image.apply_affine_transform(
    image.numpy().reshape([256, 256]), theta=0, tx = ty, ty = tx, shear=0, zx=1, zy=1, row_axis=0, col_axis=1,
    channel_axis=2, fill_mode='nearest', cval=0.0, order=1)
    
    return image, convert_target(pathname, cible)[1]
